# <span style="font-family:Courier New; color:#CCCCCC">**Pràctica 3: CA & Heuristics**</span>

In [1]:
import numpy as np
import random

## <span style="font-family:Courier New; color:#336666">**Generate Synthetic Forest Data**</span>

In [2]:
def write_img_file(img_file, data):
    with open(img_file, 'w') as f:
        for row in data:
            for element in row:
                f.write(str(element) + '\n')

In [3]:
def initialize_fuel_layer(rows, columns, cluster_len = 10):

    assert rows >= cluster_len and columns >= cluster_len, f'Error: insufficient cells for cluster_len = {cluster_len}'
    layer_data = np.zeros((rows, columns), dtype=int)
    #Assume 'cluster_len' clusters
    cluster_size_i = rows // cluster_len
    cluster_size_j = columns // cluster_len

    for clust_i in range(0, rows, cluster_size_i):
        for clust_j in range(0, columns, cluster_size_j):
            clust_density = random.randrange(1, 7, 2)
            #Assign FUEL layer's states
            for i in range(cluster_size_i):
                for j in range(cluster_size_j):
                    #Assert we don't go out of the grid
                    if clust_i + i < rows and clust_j + j < columns:
                        layer_data[clust_i+i][clust_j+j] = clust_density*random.randint(5, 10)

    #Add a fireproof diagonal path
    disp = random.randint(0, columns//2)
    for i in range(rows // 2):
        layer_data[i, i+disp] = 0

    return layer_data

In [4]:
def initialize_humidity_layer(rows, columns, cluster_len = 10):

    assert rows >= cluster_len and columns >= cluster_len, f'Error: insufficient cells for cluster_len = {cluster_len}'
    layer_data = np.zeros((rows, columns), dtype=int)
    #Assume 'cluster_len' clusters
    cluster_size_i = rows // cluster_len
    cluster_size_j = columns // cluster_len

    for clust_i in range(0, rows, cluster_size_i):
        for clust_j in range(0, columns, cluster_size_j):
            clust_density = random.randrange(1, 4)
            #Assign FUEL layer's states
            for i in range(cluster_size_i):
                for j in range(cluster_size_j):
                    #Assert we don't go out of the grid
                    if clust_i + i < rows and clust_j + j < columns:
                        layer_data[clust_i+i][clust_j+j] = clust_density*random.randint(2, 6)

    #Add random non-humid cells
    for _ in range((rows*columns) // 10):
        i = random.randint(0, rows - 1)
        j = random.randint(0, columns - 1)
        layer_data[i][j] = 0

    return layer_data

In [5]:
def initialize_fire_layer(rows, columns):

    layer_data = np.zeros((rows, columns), dtype=int)
    i = random.randint(0, rows - 1)
    j = random.randint(0, columns - 1)
    layer_data[i][j] = 1

    return layer_data

## <span style="font-family:Courier New; color:#336666">**Forest Fire CA model**</span>

In [11]:
class ForestFireCA:

    def __init__(self, rows = 100, columns = 100, random_state = 42):
        
        self._num_layers = 3
        self._rows = rows
        self._columns = columns
        self.state = self._initialize_data(random_state=random_state)


    def _initialize_data(self, random_state):

        random.seed(random_state)
        initial_state = list()
        initial_state.append(initialize_fire_layer(self._rows, self._columns))
        initial_state.append(initialize_fuel_layer(self._rows, self._columns))
        initial_state.append(initialize_humidity_layer(self._rows, self._columns))
        
        layers = ['fire', 'fuel', 'humidity']
        for i, layer in enumerate(layers):
            write_img_file(f'Initialize_{layer}.img', initial_state[i])
        return np.array(initial_state)


    def get_hood_info(self, layer, i, j):
        """
        Method to get the 'layer's states of the (i, j)'s Moore Neighborhood cells
        """

        hood_states = []
        for y in range(-1, 2):
            for x in range(-1, 2):
                if x == 0 and y == 0:
                    continue
                ni = (i + y) if (i + y < self._rows) and (i + y >= 0) else None # Closed-Boundary Assumption
                nj = (j + x) if (j + x < self._columns) and (j + x >= 0) else None
                if ni and nj: hood_states.append(self.state[layer, ni, nj])
        return hood_states
    

    def evolve(self):
        """
        Method to evolve bi-dimensional cellular automaton from time (t) to (t + 1)
        """

        self._new_state = np.zeros((self._num_layers, self._rows, self._columns), dtype = int)
        self._evolve_humidity()
        self._evolve_fuel()
        self._evolve_fire()
        self.state = self._new_state
    
    
    def _evolve_humidity(self):
        """
        Method to update humidity's layer from time (t) to (t + 1)
        """

        for i in range(self._rows):
            for j in range(self._columns):

                cell_humidity = self.state[2, i, j]
                hood_fire = self.get_hood_info(0, i, j)
                self._new_state[2, i, j] = self._humidity_func(cell_humidity, hood_fire)
    

    def _evolve_fuel(self):
        """
        Method to update fuel's layer from time (t) to (t + 1)
        """

        for i in range(self._rows):
            for j in range(self._columns):

                cell_fire, cell_fuel, cell_humidity = self.state[:, i, j]
                hood_fire = self.get_hood_info(0, i, j)
                self._new_state[0, i, j] = self._fire_func(cell_fire, cell_fuel, cell_humidity, hood_fire)


    def _evolve_fire(self):
        """
        Method to update fire's layer from time (t) to (t + 1)
        """

        for i in range(self._rows):
                for j in range(self._columns):

                    cell_fire, cell_fuel, _ = self.state[:, i, j]  
                    self._new_state[1, i, j] = self._fuel_func(cell_fire, cell_fuel)   
        

    def simulate(self, t, layer = 'fire'):
        """
        Method to carry out a simulation of t units of time

        Return: list with the bi-dimensional states of each generation step
        """
        layer_map = {'fire': 0, 'fuel': 1, 'humidity': 2}
        evolution_steps = [self.state[layer_map[layer]].copy()]
        for _ in range(t):
            self.evolve()
            evolution_steps.append(self.state[layer_map[layer]].copy())
        return evolution_steps
    

    def _fire_func(self, cell_fire, cell_fuel, cell_humidity, hood_fire):
        """
        Method to calculate the next state of a fire's layer cell, combining information from multiple current layers

        Return: -1 if Burned, 0 if UnBurned or 1 if Burning
        """

        nghbrs_burning = sum(1 for state in hood_fire if state == 1)
        if cell_fire == -1: return -1
        elif cell_fire == 0:
            if nghbrs_burning > 0 and cell_fuel > 0 and cell_humidity == 0:
                return 1
            else:
                return 0
        elif cell_fire == 1:
            if cell_fuel > 1: return 1   
            else: return 2
              

    def _fuel_func(self, cell_fire, cell_fuel):
        """
        Method to calculate the next state of a fuel's layer cell, combining information from multiple current layers

        Return: number of hours that a cell can remain in Burning state
        """
        res = cell_fuel - 1 if cell_fire == 1 else cell_fuel
        return res if res > 0 else 0


    def _humidity_func(self, cell_humidity, hood_fire):
        """
        Method to calculate the next state of a humidity's layer cell, combining information from multiple current layers

        Return: number of hours that a cell can delay its Unburned state, when hood_fire is active
        """

        nghbrs_fire = sum(1 for state in hood_fire if state == 1)
        reduce_t = (1.5* np.log(nghbrs_fire)) + 1 if nghbrs_fire > 0 else 0 #Humity reduction caused by active hood fire
        res = cell_humidity - reduce_t
        return res if res > 0 else 0

In [12]:
forest_fire = ForestFireCA()
steps = forest_fire.simulate(t = 100)

In [18]:
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(8, 8))
cmap_custom = ListedColormap(['white', 'orange', 'black'])

def update(i):
    plt.imshow(steps[i], cmap=cmap_custom, interpolation='nearest')
    ax.set_axis_off()

anim = FuncAnimation(fig, update, frames=range(len(steps)), interval=200)
anim.save('colour_rotation.gif', dpi=80)
plt.close()

MovieWriter ffmpeg unavailable; using Pillow instead.
